# Introduction

Download data from NREL End-Use Load Profiler (EULP) project.

This notebook downloads data from NREL's AWS data storage ResStock release 2022. 

1. Choose the state and weather year (amy2018, amy2012 or tmy3) to download.

2. Set up directory paths, urls, and file names.

3. Download ResStock data:

    (1) Metadata for all the building models in the ResStock dataset.

    (2) Metadata and annual results for all the building models in the state.

    (3) Time series aggregates for all the building models in the state.
    
    (4) Time series aggregates for all the building models in the ISO/RTO region.


# Packages

In [1]:
import os
import numpy as np
import pandas as pd
import eulp_downloader.downloader as dl
import requests


# Parameter setup

In [2]:
# Choose the state of study, for example 'ny' or 'ca'
state = 'ny'
# state = 'ca'

# Create directories to store the data
data_dir = '/mnt/data1/EULP/2022/'
resstock_amy2018_dir = os.path.join(data_dir, 'resstock_amy2018_release_1')
resstock_amy2012_dir = os.path.join(data_dir, 'resstock_amy2012_release_1')
resstock_tmy3_dir = os.path.join(data_dir, 'resstock_tmy3_release_1')


In [3]:
# Setup urls
url_root = 'https://oedi-data-lake.s3.amazonaws.com/nrel-pds-building-stock/end-use-load-profiles-for-us-building-stock/2022/'
resstock_amy2018_pfx = url_root + 'resstock_amy2018_release_1/'
resstock_amy2012_pfx = url_root + 'resstock_amy2012_release_1/'
resstock_tmy3_pfx = url_root + 'resstock_tmy3_release_1/'

# List of upgrades
upgrade_list = ['upgrade%02d' %x for x in range(1, 11)]
upgrade_list.insert(0, 'baseline')

upgrade_list_num = ['upgrade=%d' %x for x in range(0, 11)]


# Download metadata

In [9]:
# Create local dir if not already exists
local_dir = os.path.join(resstock_amy2018_dir, 'metadata')

if not os.path.isdir(local_dir):
    os.makedirs(local_dir)
    print('Created directory:', local_dir)
else:
    print('Directory already exists:', local_dir)

# URL for metadata
url_metadata = resstock_amy2018_pfx + 'metadata/'

# Loop through location id and building bldg_type and download data
local_filename_list = list()
failed_filename_list = list()

for upgrade in upgrade_list:
    filename = upgrade + '.parquet'
    local_filename = os.path.join(local_dir, filename)
    local_filename_list.append(local_filename)

    # Download the file if not already exists
    if not os.path.isfile(local_filename):
        req = requests.get(url_metadata + filename)
        
        # If the request is successful
        if req.status_code == 200:
            url_content = req.content

            # Write the content to a local file
            with open(local_filename, 'wb') as file:
                file.write(url_content)
            print('Downloaded:', filename)
                
        else:
            failed_filename_list.append(local_filename)
            print('File cannot be found:', filename)



Directory already exists: /mnt/data1/EULP/2022/resstock_amy2018_release_1/metadata


In [6]:
metadata_df = pd.read_parquet(local_filename_list[0], engine='pyarrow')
metadata_df

,applicability,in.sqft,weight,in.ahs_region,in.ashrae_iecc_climate_zone_2004,in.ashrae_iecc_climate_zone_2004_2_a_split,in.bathroom_spot_vent_hour,in.bedrooms,in.building_america_climate_zone,in.cec_climate_zone,...,out.electricity.total.energy_consumption,out.electricity.total.energy_consumption_intensity,out.fuel_oil.total.energy_consumption,out.fuel_oil.total.energy_consumption_intensity,out.natural_gas.total.energy_consumption,out.natural_gas.total.energy_consumption_intensity,out.propane.total.energy_consumption,out.propane.total.energy_consumption_intensity,upgrade,metadata_index
bldg_id,,,,,,,,,,,,,,,,,,,,,
1,True,1690.0,242.131013,Non-CBSA South Atlantic,4A,4A,Hour22,3,Mixed-Humid,None,...,9505.467084,5.624537,0.0,0.0,0.000000,0.000000,16516.313221,9.772966,0,0
2,True,1690.0,242.131013,Non-CBSA Mountain,6B,6B,Hour22,3,Cold,None,...,5695.836245,3.370317,0.0,0.0,51311.762149,30.361989,0.000000,0.000000,0,1
4,True,2176.0,242.131013,Non-CBSA West South Central,2A,"2A - TX, LA",Hour7,4,Hot-Humid,None,...,12548.717075,5.766874,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0,2
5,True,2176.0,242.131013,Non-CBSA West North Central,4A,4A,Hour19,3,Mixed-Humid,None,...,28793.646485,13.232374,0.0,0.0,4917.146412,2.259718,0.000000,0.000000,0,3
6,True,1690.0,242.131013,"CBSA Miami-Fort Lauderdale-West Palm Beach, FL",2A,"2A - FL, GA, AL, MS",Hour7,3,Hot-Humid,None,...,9921.628004,5.870786,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549996,True,853.0,242.131013,"CBSA Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",4A,4A,Hour0,2,Mixed-Humid,None,...,11320.749292,13.271687,0.0,0.0,0.000000,0.000000,555.955820,0.651765,0,548911
549997,True,1623.0,242.131013,"CBSA New York-Newark-Jersey City, NY-NJ-PA",5A,5A,Hour8,3,Cold,None,...,6339.127244,3.905809,0.0,0.0,25467.289841,15.691491,0.000000,0.000000,0,548912
549998,True,1690.0,242.131013,"CBSA Los Angeles-Long Beach-Anaheim, CA",3B,3B,Hour3,2,Hot-Dry,8,...,11987.779047,7.093360,0.0,0.0,9769.524118,5.780784,0.000000,0.000000,0,548913


In [7]:
metadata_df.columns

Index(['applicability', 'in.sqft', 'weight', 'in.ahs_region',
       'in.ashrae_iecc_climate_zone_2004',
       'in.ashrae_iecc_climate_zone_2004_2_a_split',
       'in.bathroom_spot_vent_hour', 'in.bedrooms',
       'in.building_america_climate_zone', 'in.cec_climate_zone',
       ...
       'out.electricity.total.energy_consumption',
       'out.electricity.total.energy_consumption_intensity',
       'out.fuel_oil.total.energy_consumption',
       'out.fuel_oil.total.energy_consumption_intensity',
       'out.natural_gas.total.energy_consumption',
       'out.natural_gas.total.energy_consumption_intensity',
       'out.propane.total.energy_consumption',
       'out.propane.total.energy_consumption_intensity', 'upgrade',
       'metadata_index'],
      dtype='object', length=272)

# Download metadata and annual result

In [35]:
# There are two file types for the annual results: parquet and csv
# Choose the file type to download
# Parquet is faster to read, but csv is easier to view
# format = 'parquet'
format = 'csv'

# Create local dir if not already exists
local_dir = os.path.join(resstock_amy2018_dir, 
                        'metadata_and_annual_results',
                        'by_state',
                        f'state={state.upper()}',
                        format)

if not os.path.isdir(local_dir):
    os.makedirs(local_dir)
    print('Created directory:', local_dir)
else:
    print('Directory already exists:', local_dir)

# URL for metadata and annual results (state specific)
url_metadata = resstock_amy2018_pfx + 'metadata_and_annual_results/' \
    + f'by_state/state={state.upper()}/{format}/'

# Loop through location id and building bldg_type and download data
local_filename_list = list()
failed_filename_list = list()

for upgrade in upgrade_list:
    filename = f'{state.upper()}_{upgrade}_metadata_and_annual_results.{format}'
    local_filename = os.path.join(local_dir, filename)
    local_filename_list.append(local_filename)

    # Download the file if not already exists
    if not os.path.isfile(local_filename):
        req = requests.get(url_metadata + filename)

        # If the request is successful
        if req.status_code == 200:
            url_content = req.content

            # Write the content to a local file
            with open(local_filename, 'wb') as file:
                file.write(url_content)
            print('Downloaded:', filename)
            
        else:
            failed_filename_list.append(local_filename)
            print('File cannot be found:', filename)


Directory already exists: /mnt/data1/EULP/2022/resstock_amy2018_release_1/metadata_and_annual_results/by_state/state=NY/csv


In [36]:
if format == 'parquet':
    metadata_and_annual_result_df = pd.read_parquet(local_filename_list[0], 
                                                engine='pyarrow')
else:
    metadata_and_annual_result_df = pd.read_csv(local_filename_list[0])
    metadata_and_annual_result_df.set_index('bldg_id', inplace=True)

metadata_and_annual_result_df

,upgrade,weight,applicability,in.sqft,in.ahs_region,in.ashrae_iecc_climate_zone_2004,in.ashrae_iecc_climate_zone_2004_2_a_split,in.bathroom_spot_vent_hour,in.bedrooms,in.building_america_climate_zone,...,out.emissions.natural_gas.lrmer_low_re_cost_25_2025_start.co2e_kg,out.emissions.propane.lrmer_low_re_cost_25_2025_start.co2e_kg,out.emissions.electricity.lrmer_mid_case_15_2025_start.co2e_kg,out.emissions.fuel_oil.lrmer_mid_case_15_2025_start.co2e_kg,out.emissions.natural_gas.lrmer_mid_case_15_2025_start.co2e_kg,out.emissions.propane.lrmer_mid_case_15_2025_start.co2e_kg,out.emissions.all_fuels.lrmer_95_decarb_by_2035_15_2025_start.co2e_kg,out.emissions.all_fuels.lrmer_low_re_cost_15_2025_start.co2e_kg,out.emissions.all_fuels.lrmer_low_re_cost_25_2025_start.co2e_kg,out.emissions.all_fuels.lrmer_mid_case_15_2025_start.co2e_kg
bldg_id,,,,,,,,,,,,,,,,,,,,,
10,0,242.131013,True,885.0,Non-CBSA Middle Atlantic,5A,5A,Hour22,1,Cold,...,7272.946781,0.000000,1279.073331,0.0,7272.946781,0.000000,8314.635266,8712.701580,8506.977935,8552.020111
11,0,242.131013,True,1138.0,"CBSA New York-Newark-Jersey City, NY-NJ-PA",4A,4A,Hour0,2,Mixed-Humid,...,4523.731137,0.000000,641.740217,0.0,4523.731137,0.000000,5056.370596,5245.619312,5143.569647,5165.471354
27,0,242.131013,True,1623.0,"CBSA New York-Newark-Jersey City, NY-NJ-PA",4A,4A,Hour15,2,Mixed-Humid,...,0.000000,0.000000,1401.335525,0.0,0.000000,0.000000,1160.504739,1558.880856,1334.964529,1401.335525
64,0,242.131013,True,853.0,"CBSA New York-Newark-Jersey City, NY-NJ-PA",4A,4A,Hour22,2,Mixed-Humid,...,5797.469768,0.000000,820.839804,0.0,5797.469768,0.000000,6486.016182,6715.016826,6586.975864,6618.309572
67,0,242.131013,True,2663.0,Non-CBSA Middle Atlantic,5A,5A,Hour8,5,Cold,...,0.000000,7402.508184,2710.206246,0.0,0.000000,7402.508184,9614.489931,10416.960605,9962.438370,10112.714430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549854,0,242.131013,True,853.0,"CBSA New York-Newark-Jersey City, NY-NJ-PA",4A,4A,Hour4,1,Mixed-Humid,...,2170.693956,0.000000,622.135501,0.0,2170.693956,0.000000,2724.276681,2874.508742,2799.103547,2792.829457
549873,0,242.131013,True,853.0,"CBSA New York-Newark-Jersey City, NY-NJ-PA",4A,4A,Hour6,2,Mixed-Humid,...,2454.202795,0.000000,601.211285,0.0,2454.202795,0.000000,2955.860987,3128.915095,3033.406685,3055.414080
549906,0,242.131013,True,2663.0,Non-CBSA Middle Atlantic,5A,5A,Hour3,4,Cold,...,422.560755,0.000000,768.973330,0.0,422.560755,0.000000,1117.366290,1278.600234,1199.439747,1191.534086


In [25]:
metadata_and_annual_result_df.columns

Index(['bldg_id', 'upgrade', 'weight', 'applicability', 'in.sqft',
       'in.ahs_region', 'in.ashrae_iecc_climate_zone_2004',
       'in.ashrae_iecc_climate_zone_2004_2_a_split',
       'in.bathroom_spot_vent_hour', 'in.bedrooms',
       ...
       'out.emissions.natural_gas.lrmer_low_re_cost_25_2025_start.co2e_kg',
       'out.emissions.propane.lrmer_low_re_cost_25_2025_start.co2e_kg',
       'out.emissions.electricity.lrmer_mid_case_15_2025_start.co2e_kg',
       'out.emissions.fuel_oil.lrmer_mid_case_15_2025_start.co2e_kg',
       'out.emissions.natural_gas.lrmer_mid_case_15_2025_start.co2e_kg',
       'out.emissions.propane.lrmer_mid_case_15_2025_start.co2e_kg',
       'out.emissions.all_fuels.lrmer_95_decarb_by_2035_15_2025_start.co2e_kg',
       'out.emissions.all_fuels.lrmer_low_re_cost_15_2025_start.co2e_kg',
       'out.emissions.all_fuels.lrmer_low_re_cost_25_2025_start.co2e_kg',
       'out.emissions.all_fuels.lrmer_mid_case_15_2025_start.co2e_kg'],
      dtype='object', le

# Download time series aggregates

In [18]:
# ResStock building types
resstock_type_list = [
    'mobile_home',
    'multi-family_with_2_-_4_units',
    'multi-family_with_5plus_units',
    'single-family_attached',
    'single-family_detached'
]

## Time series aggregates by state

In [26]:
# Loop through upgrade types
for i, upgrade_num in enumerate(upgrade_list_num):
    upgrade_num = upgrade_list_num[i]

    # Create local dir if not already exists
    local_dir = os.path.join(resstock_amy2018_dir, 
                            'timeseries_aggregates',
                            'by_state',
                            upgrade_num,
                            f'state={state.upper()}')

    if not os.path.isdir(local_dir):
        os.makedirs(local_dir)
        print('Created directory:', local_dir)
    else:
        print('Directory already exists:', local_dir)

    # URL for metadata and annual results (state specific)
    url_pfx = resstock_amy2018_pfx + 'timeseries_aggregates/' \
        + f'by_state/{upgrade_num}/state={state.upper()}/'

    # Loop through location id and building bldg_type and download data
    local_filename_list = list()
    failed_filename_list = list()

    for bldg_type in resstock_type_list:
        filename = 'up%02d' %i + f'-{state}-{bldg_type}.csv'
        local_filename = os.path.join(local_dir, filename)
        local_filename_list.append(local_filename)

        # Download the file if not already exists
        if not os.path.isfile(local_filename):
            req = requests.get(url_pfx + filename)

            # If the request is successful
            if req.status_code == 200:
                url_content = req.content

                # Write the content to a local file
                with open(local_filename, 'wb') as file:
                    file.write(url_content)
                print('Downloaded:', filename)

            else:
                failed_filename_list.append(local_filename)
                print('File cannot be found:', filename)


Directory already exists: /mnt/data1/EULP/2022/resstock_amy2018_release_1/timeseries_aggregates/by_state/upgrade=0/state=NY
Directory already exists: /mnt/data1/EULP/2022/resstock_amy2018_release_1/timeseries_aggregates/by_state/upgrade=1/state=NY
Directory already exists: /mnt/data1/EULP/2022/resstock_amy2018_release_1/timeseries_aggregates/by_state/upgrade=2/state=NY
Directory already exists: /mnt/data1/EULP/2022/resstock_amy2018_release_1/timeseries_aggregates/by_state/upgrade=3/state=NY
Directory already exists: /mnt/data1/EULP/2022/resstock_amy2018_release_1/timeseries_aggregates/by_state/upgrade=4/state=NY
Directory already exists: /mnt/data1/EULP/2022/resstock_amy2018_release_1/timeseries_aggregates/by_state/upgrade=5/state=NY
Directory already exists: /mnt/data1/EULP/2022/resstock_amy2018_release_1/timeseries_aggregates/by_state/upgrade=6/state=NY
Directory already exists: /mnt/data1/EULP/2022/resstock_amy2018_release_1/timeseries_aggregates/by_state/upgrade=7/state=NY
Director

In [27]:
ts_agg_state_df = pd.read_csv(local_filename)
ts_agg_state_df

,upgrade,in.state,in.geometry_building_type_recs,timestamp,models_used,units_represented,out.electricity.ceiling_fan.energy_consumption.kwh,out.electricity.clothes_dryer.energy_consumption.kwh,out.electricity.clothes_washer.energy_consumption.kwh,out.electricity.cooling_fans_pumps.energy_consumption.kwh,...,out.site_energy.total.energy_consumption.kwh.savings,out.electricity.net.energy_consumption.kwh.savings,out.electricity.total.energy_consumption.kwh.savings,out.fuel_oil.total.energy_consumption.kwh.savings,out.natural_gas.total.energy_consumption.kwh.savings,out.propane.total.energy_consumption.kwh.savings,out.emissions_reduction.all_fuels.lrmer_95decarbby2035_15_2025start.co2e_kg,out.emissions_reduction.all_fuels.lrmer_lowrecost_15_2025start.co2e_kg,out.emissions_reduction.all_fuels.lrmer_lowrecost_25_2025start.co2e_kg,out.emissions_reduction.all_fuels.lrmer_midcase_15_2025start.co2e_kg
0,10,NY,Single-Family Detached,2018-01-01 00:15:00,14131,3.421553e+06,2906.056415,12910.425599,2546.491861,0.000000,...,9.571320e+06,-43555.979355,-2.617684e+06,3.645373e+06,7.941162e+06,602438.820977,2.624170e+06,2.369210e+06,2.514490e+06,2.491716e+06
1,10,NY,Single-Family Detached,2018-01-01 00:30:00,14131,3.421553e+06,2859.567260,13026.890616,2869.978894,0.000000,...,1.006849e+07,-45268.814139,-2.683135e+06,3.808902e+06,8.308506e+06,634187.961072,2.754801e+06,2.493466e+06,2.642379e+06,2.619036e+06
2,10,NY,Single-Family Detached,2018-01-01 00:45:00,14131,3.421553e+06,2772.400096,16524.473095,2153.997489,0.000000,...,1.027556e+07,-48907.558999,-2.904826e+06,3.937758e+06,8.583130e+06,659482.079679,2.824709e+06,2.541780e+06,2.702997e+06,2.677725e+06
3,10,NY,Single-Family Detached,2018-01-01 01:00:00,14131,3.421553e+06,2738.259623,20781.862691,1281.115188,0.000000,...,1.052726e+07,-52869.548760,-3.180806e+06,4.098076e+06,8.921424e+06,688541.917248,2.910224e+06,2.600415e+06,2.776949e+06,2.749276e+06
4,10,NY,Single-Family Detached,2018-01-01 01:15:00,14131,3.421553e+06,2635.596074,20855.228388,762.470559,1.694917,...,1.045715e+07,-54181.414587,-3.138260e+06,4.069422e+06,8.842132e+06,683863.845060,2.887404e+06,2.588642e+06,2.760305e+06,2.737395e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35035,10,NY,Single-Family Detached,2018-12-31 23:00:00,14131,3.421553e+06,4842.862386,32397.613765,1628.815323,0.726393,...,5.552633e+06,-7458.361585,-4.505625e+05,1.783371e+06,3.907777e+06,312031.757754,1.424030e+06,1.379606e+06,1.404296e+06,1.398259e+06
35036,10,NY,Single-Family Detached,2018-12-31 23:15:00,14131,3.421553e+06,3958.842058,30257.902005,1225.425055,15.254254,...,5.266982e+06,-7907.272483,-4.291666e+05,1.699148e+06,3.705525e+06,291468.861569,1.354467e+06,1.312924e+06,1.335541e+06,1.333782e+06
35037,10,NY,Single-Family Detached,2018-12-31 23:30:00,14131,3.421553e+06,3812.352795,26242.159159,1260.049790,3.147703,...,5.267370e+06,-7471.436660,-4.374450e+05,1.705834e+06,3.706467e+06,292499.365851,1.355341e+06,1.312998e+06,1.336051e+06,1.334257e+06
35038,10,NY,Single-Family Detached,2018-12-31 23:45:00,14131,3.421553e+06,3527.848855,23678.960259,1013.076157,5.084751,...,5.164197e+06,-7537.538426,-4.294051e+05,1.668224e+06,3.633764e+06,291608.372064,1.328721e+06,1.287155e+06,1.309784e+06,1.308024e+06


In [28]:
ts_agg_state_df.columns

Index(['upgrade', 'in.state', 'in.geometry_building_type_recs', 'timestamp',
       'models_used', 'units_represented',
       'out.electricity.ceiling_fan.energy_consumption.kwh',
       'out.electricity.clothes_dryer.energy_consumption.kwh',
       'out.electricity.clothes_washer.energy_consumption.kwh',
       'out.electricity.cooling_fans_pumps.energy_consumption.kwh',
       ...
       'out.site_energy.total.energy_consumption.kwh.savings',
       'out.electricity.net.energy_consumption.kwh.savings',
       'out.electricity.total.energy_consumption.kwh.savings',
       'out.fuel_oil.total.energy_consumption.kwh.savings',
       'out.natural_gas.total.energy_consumption.kwh.savings',
       'out.propane.total.energy_consumption.kwh.savings',
       'out.emissions_reduction.all_fuels.lrmer_95decarbby2035_15_2025start.co2e_kg',
       'out.emissions_reduction.all_fuels.lrmer_lowrecost_15_2025start.co2e_kg',
       'out.emissions_reduction.all_fuels.lrmer_lowrecost_25_2025start.co2e_k

## Time series aggregates by ISO

In [29]:
iso_name = 'nyiso'

# Loop through upgrade types
for i, upgrade_num in enumerate(upgrade_list_num):
    upgrade_num = upgrade_list_num[i]

    # Create local dir if not already exists
    local_dir = os.path.join(resstock_amy2018_dir, 
                            'timeseries_aggregates',
                            'by_iso_rto_region',
                            upgrade_num,
                            f'iso_rto_region={iso_name.upper()}')

    if not os.path.isdir(local_dir):
        os.makedirs(local_dir)
        print('Created directory:', local_dir)
    else:
        print('Directory already exists:', local_dir)

    # URL for metadata and annual results (state specific)
    url_pfx = resstock_amy2018_pfx + 'timeseries_aggregates/' \
        + f'by_iso_rto_region/{upgrade_num}/iso_rto_region={iso_name.upper()}/'

    # Loop through location id and building bldg_type and download data
    local_filename_list = list()
    failed_filename_list = list()

    for bldg_type in resstock_type_list:
        filename = 'up%02d' %i + f'-{iso_name}-{bldg_type}.csv'
        local_filename = os.path.join(local_dir, filename)
        local_filename_list.append(local_filename)

        # Download the file if not already exists
        if not os.path.isfile(local_filename):
            req = requests.get(url_pfx + filename)

            # If the request is successful
            if req.status_code == 200:
                url_content = req.content

                # Write the content to a local file
                with open(local_filename, 'wb') as file:
                    file.write(url_content)
                print('Downloaded:', filename)

            else:
                failed_filename_list.append(local_filename)
                print('File cannot be found:', filename)


Directory already exists: /mnt/data1/EULP/2022/resstock_amy2018_release_1/timeseries_aggregates/by_iso_rto_region/upgrade=0/iso_rto_region=NYISO
Directory already exists: /mnt/data1/EULP/2022/resstock_amy2018_release_1/timeseries_aggregates/by_iso_rto_region/upgrade=1/iso_rto_region=NYISO
Directory already exists: /mnt/data1/EULP/2022/resstock_amy2018_release_1/timeseries_aggregates/by_iso_rto_region/upgrade=2/iso_rto_region=NYISO
Directory already exists: /mnt/data1/EULP/2022/resstock_amy2018_release_1/timeseries_aggregates/by_iso_rto_region/upgrade=3/iso_rto_region=NYISO
Directory already exists: /mnt/data1/EULP/2022/resstock_amy2018_release_1/timeseries_aggregates/by_iso_rto_region/upgrade=4/iso_rto_region=NYISO
Directory already exists: /mnt/data1/EULP/2022/resstock_amy2018_release_1/timeseries_aggregates/by_iso_rto_region/upgrade=5/iso_rto_region=NYISO
Directory already exists: /mnt/data1/EULP/2022/resstock_amy2018_release_1/timeseries_aggregates/by_iso_rto_region/upgrade=6/iso_rt

# Individual building time series

In [37]:
bldg_id_list = metadata_and_annual_result_df.index.to_list()

failed_filename_list = list()

# Loop through upgrade types
for i, upgrade_num in enumerate(upgrade_list_num):
    upgrade_num = upgrade_list_num[i]

    # Create local dir if not already exists
    local_dir = os.path.join(resstock_amy2018_dir, 
                            'timeseries_individual_buildings',
                            'by_state',
                            upgrade_num,
                            f'state={state.upper()}')

    if not os.path.isdir(local_dir):
        os.makedirs(local_dir)
        print('Created directory:', local_dir)
    else:
        print('Directory already exists:', local_dir)

    # URL for metadata and annual results (state specific)
    url_pfx = resstock_amy2018_pfx + 'timeseries_individual_buildings/' \
        + f'by_state/{upgrade_num}/state={state.upper()}/'

    local_filename_list = list()

    # Loop through building ids
    for bldg_id in bldg_id_list:
        filename = f'{bldg_id}-{i}.parquet'
        local_filename = os.path.join(local_dir, filename)
        local_filename_list.append(local_filename)

        # Download the file if not already exists
        if not os.path.isfile(local_filename):
            req = requests.get(url_pfx + filename)

            # If the request is successful
            if req.status_code == 200:
                url_content = req.content

                # Write the content to a local file
                with open(local_filename, 'wb') as file:
                    file.write(url_content)
                print('Downloaded:', filename)

            else:
                failed_filename_list.append(local_filename)
                print('File cannot be found:', filename)

Directory already exists: /mnt/data1/EULP/2022/resstock_amy2018_release_1/timeseries_individual_buildings/by_state/upgrade=0/state=NY
Directory already exists: /mnt/data1/EULP/2022/resstock_amy2018_release_1/timeseries_individual_buildings/by_state/upgrade=1/state=NY
Directory already exists: /mnt/data1/EULP/2022/resstock_amy2018_release_1/timeseries_individual_buildings/by_state/upgrade=2/state=NY
Directory already exists: /mnt/data1/EULP/2022/resstock_amy2018_release_1/timeseries_individual_buildings/by_state/upgrade=3/state=NY
Directory already exists: /mnt/data1/EULP/2022/resstock_amy2018_release_1/timeseries_individual_buildings/by_state/upgrade=4/state=NY
Directory already exists: /mnt/data1/EULP/2022/resstock_amy2018_release_1/timeseries_individual_buildings/by_state/upgrade=5/state=NY
Directory already exists: /mnt/data1/EULP/2022/resstock_amy2018_release_1/timeseries_individual_buildings/by_state/upgrade=6/state=NY
Directory already exists: /mnt/data1/EULP/2022/resstock_amy201